In [19]:
from KmeansPPcenters import KMeanPlusPlus
from Gonzalez_centers import Gonzalez
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from numpy import genfromtxt


In [20]:
dirNam = "realData/Skin/"
proDir = "realDataProcessed/"

#Processing skin
fileName = "Skin_NonSkin.txt"

skin = np.genfromtxt(dirNam + fileName, delimiter='')

np.savetxt(proDir + "skin.csv", skin, delimiter=",",  fmt='%f')

In [24]:
skin_data = genfromtxt('realDataProcessed/skin.csv', delimiter=',')
skin_data=skin_data[:,0:3]
skin_data


array([[ 74.,  85., 123.],
       [ 73.,  84., 122.],
       [ 72.,  83., 121.],
       ...,
       [163., 162., 112.],
       [163., 162., 112.],
       [255., 255., 255.]])

In [27]:
%%time
data= skin_data
num_clusters= 5
init=data[0]

#Get KPP centers
KPP=KMeanPlusPlus(num_clusters=num_clusters, init=init)
KPP.fit(data)
KPP_centers= KPP.centers
print(KPP_centers)

#Get Gonzalez centers
#GC_centers = Gonzalez(data, num_clusters=num_clusters, init=init)
#print(GC_centers)

[[ 74.  85. 123.]
 [164. 162. 114.]
 [106.  67.  53.]
 [166. 180. 232.]
 [ 17.  14.   0.]]
CPU times: user 245 ms, sys: 3.99 ms, total: 249 ms
Wall time: 250 ms


In [18]:
%%time
num_clusters=3
data= skin_data

#With Gonzalez intialization
kmeans = KMeans(n_clusters=num_clusters, init=GC_centers)
# Fitting the input data
kmeans = kmeans.fit(data)

# Centroid values
centroids = kmeans.cluster_centers_
print("Gconzalez: {}".format(centroids))

#With KMeans++ intialization
kmeans = KMeans(n_clusters=num_clusters, init="KPP_centers")
# Fitting the input data
kmeans = kmeans.fit(data)

# Centroid values
centroids = kmeans.cluster_centers_
print("KPP: {}".format(centroids))

#With random intialization
kmeans = KMeans(n_clusters=num_clusters, init="random")
# Fitting the input data
kmeans = kmeans.fit(data)

# Centroid values
centroids = kmeans.cluster_centers_
print("Random: {}".format(centroids))

ValueError: the init parameter for the k-means should be 'k-means++' or 'random' or an ndarray, 'GC_centers' (type '<class 'str'>') was passed.